In [3]:
import cv2
import numpy as np

image = cv2.imread('pic27c.jpg')
kernel = np.array([[-1,-1,-1], 
                   [-1, 9,-1],
                   [-1,-1,-1]])
sharpened = cv2.filter2D(image, -1, kernel)
cv2.imwrite("pic27sh.jpg",sharpened)

True

In [12]:
img1 = cv2.imread('pic27c.jpg',0)
blur = cv2.GaussianBlur(img1,(5,5),0)
denoise= cv2.fastNlMeansDenoising(blur,None,7,21)
cv2.imwrite("pic27dn.jpg",denoise)

True

In [14]:
img=denoise
img = cv2.medianBlur(img,5)
th= cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
cv2.imwrite("pic27dnat.jpg",th)

True

In [8]:
print(blur)

None


In [9]:
print(hi)

NameError: name 'hi' is not defined

In [1]:
from __future__ import print_function
import cv2 as cv
import numpy as np
import argparse
import random as rng
src = cv.imread("pic27c.jpg")
src[np.all(src == 255, axis=2)] = 0
# Show output image
kernel = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]], dtype=np.float32)
# do the laplacian filtering as it is
# well, we need to convert everything in something more deeper then CV_8U
# because the kernel has some negative values,
# and we can expect in general to have a Laplacian image with negative values
# BUT a 8bits unsigned int (the one we are working with) can contain values from 0 to 255
# so the possible negative number will be truncated
imgLaplacian = cv.filter2D(src, cv.CV_32F, kernel)
sharp = np.float32(src)
imgResult = sharp - imgLaplacian
# convert back to 8bits gray scale
imgResult = np.clip(imgResult, 0, 255)
imgResult = imgResult.astype('uint8')
imgLaplacian = np.clip(imgLaplacian, 0, 255)
imgLaplacian = np.uint8(imgLaplacian)
#cv.imshow('Laplace Filtered Image', imgLaplacian)
bw = cv.cvtColor(imgResult, cv.COLOR_BGR2GRAY)
_, bw = cv.threshold(bw, 40, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
dist = cv.distanceTransform(bw, cv.DIST_L2, 3)
# Normalize the distance image for range = {0.0, 1.0}
# so we can visualize and threshold it
cv.normalize(dist, dist, 0, 1.0, cv.NORM_MINMAX)
_, dist = cv.threshold(dist, 0.4, 1.0, cv.THRESH_BINARY)
# Dilate a bit the dist image
kernel1 = np.ones((3,3), dtype=np.uint8)
dist = cv.dilate(dist, kernel1)
dist_8u = dist.astype('uint8')
# Find total markers
_, contours, _ = cv.findContours(dist_8u, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
# Create the marker image for the watershed algorithm
markers = np.zeros(dist.shape, dtype=np.int32)
# Draw the foreground markers
for i in range(len(contours)):
    cv.drawContours(markers, contours, i, (i+1), -1)
# Draw the background marker
cv.circle(markers, (5,5), 3, (255,255,255), -1)
cv.watershed(imgResult, markers)
#mark = np.zeros(markers.shape, dtype=np.uint8)
mark = markers.astype('uint8')
mark = cv.bitwise_not(mark)
# uncomment this if you want to see how the mark
# image looks like at that point
#cv.imshow('Markers_v2', mark)
# Generate random colors
colors = []
for contour in contours:
    colors.append((rng.randint(0,256), rng.randint(0,256), rng.randint(0,256)))
# Create the result image
dst = np.zeros((markers.shape[0], markers.shape[1], 3), dtype=np.uint8)
# Fill labeled objects with random colors
for i in range(markers.shape[0]):
    for j in range(markers.shape[1]):
        index = markers[i,j]
        if index > 0 and index <= len(contours):
            dst[i,j,:] = colors[index-1]
# Visualize the final image
cv.imwrite("pic27seg.jpg",dst)

True

In [2]:
help(cv.distanceTransform)

Help on built-in function distanceTransform:

distanceTransform(...)
    distanceTransform(src, distanceType, maskSize[, dst[, dstType]]) -> dst
    .   @overload
    .   @param src 8-bit, single-channel (binary) source image.
    .   @param dst Output image with calculated distances. It is a 8-bit or 32-bit floating-point,
    .   single-channel image of the same size as src .
    .   @param distanceType Type of distance, see #DistanceTypes
    .   @param maskSize Size of the distance transform mask, see #DistanceTransformMasks. In case of the
    .   #DIST_L1 or #DIST_C distance type, the parameter is forced to 3 because a \f$3\times 3\f$ mask gives
    .   the same result as \f$5\times 5\f$ or any larger aperture.
    .   @param dstType Type of output image. It can be CV_8U or CV_32F. Type CV_8U can be used only for
    .   the first variant of the function and distanceType == #DIST_L1.



In [3]:
img=cv.imread("pic27c.jpg")
blur = cv.GaussianBlur(img,(5,5),0)
ret,th = cv.threshold(blur,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
dist = cv.distanceTransform(th, cv.DIST_L2, 5)


error: OpenCV(3.4.1) C:\Miniconda3\conda-bld\opencv-suite_1533128839831\work\modules\imgproc\src\thresh.cpp:1406: error: (-215) src.type() == (((0) & ((1 << 3) - 1)) + (((1)-1) << 3)) in function cv::threshold


In [10]:
img=cv.imread("pic27c.jpg",0)
img1=img
blur = cv.GaussianBlur(img1,(9,9),0)
img2=img1-blur
ret,thresh = cv.threshold(img2,127,255,cv.THRESH_BINARY)
cv.imwrite("pic27cbelmn.jpg",thresh)

True

In [ ]:
# -*- encoding: windows-1251

# http://stackoverflow.com/questions/22122309/opencv-adaptive-threshold-ocr

import sys
import cv2
import numpy as np

DEFAULT_BLOCKSIZE = 21  # 
MEAN_VARIANCE_THREASHOLD = 0.01

def calc_block_mean_variance(image, mask, blocksize):
    """ image - image converted to float32 and scaled to 0..1
    """
    I = image.copy()

    result = np.zeros( (image.shape[0] / blocksize, image.shape[1] / blocksize) )
    small_image = np.zeros( (image.shape[0] / blocksize, image.shape[1] / blocksize) )

    for i in xrange(0, image.shape[0] - blocksize, blocksize):
        for j in xrange(0, image.shape[1] - blocksize, blocksize):

            patch = I[ i:i+blocksize+1, j:j+blocksize+1]
            mask_patch = mask[ i:i+blocksize+1, j:j+blocksize+1]

            tmp1 = np.zeros((blocksize, blocksize))
            tmp2 = np.zeros((blocksize, blocksize))
            mean, std_dev = cv2.meanStdDev(patch, tmp1, tmp2, mask_patch)

            value = 0
            if std_dev[0][0] > MEAN_VARIANCE_THREASHOLD:
                value = mean[0][0]

            result[i/blocksize, j/blocksize] = value


    result *= 255
    result = cv2.convertScaleAbs(result)


    small_image = cv2.resize(I, (small_image.shape[1], small_image.shape[0]))
    small_image *= 255;
    small_image = cv2.convertScaleAbs(small_image)

    # Здесь в функции inpaint есть страшный глюк при работе с маленькими блоками
    # в первом ряде могут вылазить ненужные артефакты. 
    # поэтому здесь будет страшный хак - мы копируем первый ряд дважды, а после преобразования его удаляем

    #small_image = np.insert(small_image, 0, small_image[:,0], 1)
    #result = np.insert(result, 0, result[:,0], 1)

    res, inpaintmask = cv2.threshold(result, 3, 255, cv2.THRESH_BINARY)

    inpainted = cv2.inpaint(small_image, inpaintmask, 10,  cv2.INPAINT_NS);

    #inpainted = np.delete(inpainted, 0, 1)
    #small_image = np.delete(small_image, 0, 1)

    inpainted = cv2.resize(inpainted, (image.shape[1], image.shape[0]))

    inpainted = np.float32(inpainted)
    inpainted *= 1./255.

    return inpainted


def normalize(image):

    image = cv2.resize(image, (image.shape[1] * 2, image.shape[0] * 2))
        
#    alpha = image[:, :, 3]

#    mask = alpha.copy()
#    mask[:] = 0
#    mask[alpha == 255] = 1

    # remove
    mask = image[:,:, 0]
    mask[:] = 255

    # make alpha channel white    
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)
#    image[alpha < 255] = 255

    # Convert to Float values
    image_f = np.float32(image)
    image_f *= 1./255.   

    res = calc_block_mean_variance(image_f, mask, DEFAULT_BLOCKSIZE * 4)    

    cv2.imwrite("mask.jpg", res * 255)

    res = 1.0 - res
    res = image_f + res

    res *= 255
    res = cv2.convertScaleAbs(res)

    

    _, res = cv2.threshold(res, 217, 255, cv2.THRESH_BINARY) #  THRESH_BINARY THRESH_OTSU

    res = cv2.resize(res, (image.shape[1] / 2, image.shape[0] / 2))

    return res
   

if __name__ == '__main__':

    fname = "img/11.png" if len(sys.argv) < 2 else sys.argv[1]
    outfile = "out.png" if len(sys.argv) < 3 else sys.argv[2]

    image = cv2.imread(fname, cv2.CV_LOAD_IMAGE_UNCHANGED)

    result = normalize(image)

    cv2.imwrite(outfile, result)

In [1]:
f= open("demo.txt","w+")
f.write("hi")
f.write("naren")
f.close()

In [16]:
def binaryval(var):
    if var=='?':
        v=0
    elif var==" ":
        v=0
    else:
        v= ord(var)-10240
    b=bin(v)
    b=b[2:len(b)]
    if len(b)==1:
        d="00000"+b
    elif len(b)==2:
        d="0000"+b
    elif len(b)==3:
        d="000"+b
    elif len(b)==4:
        d="00"+b
    elif len(b)==5:
        d="0"+b
    else:
        d=b
    return d

In [19]:
binaryval('⠗')

'010111'

In [18]:
ord("⠗")

10263

In [25]:
string= ⠉⠕⠗⠞⠑ ⠔ ⠏⠗⠑⠕⠗⠙⠑ ⠠⠍ ⠏⠗⠑⠉⠊⠎⠑⠇⠽⠒  ⠙ ⠊⠝⠁ ⠋⠊⠭⠑⠙ ⠁⠍ ⠁⠇⠕⠛⠕⠥⠎ ⠏⠗⠕⠏⠑⠗⠞⠽ ⠞ ⠑⠁⠉⠓ ⠧⠑⠗⠞⠑⠭  ⠎⠉⠑⠝⠙⠁⠝⠞ ⠷ ? ? ⠷ ⠞⠗⠑⠑ ⠎⠊⠵⠑⠲ ⠠⠏⠕⠎⠞⠕⠗⠙⠑⠗ ⠝⠥⠍⠖  ⠥⠍⠃⠑⠗⠎ ⠷ ? ⠃⠊⠝⠁⠛⠽?⠤⠞⠑⠑⠞  ⠠⠎⠕⠗⠁ ⠧⠑⠗⠞⠑⠭ ⠧ ⠓⠁⠎ ⠁ ⠝⠥⠖  ⠗⠊⠛⠓⠞ ⠎⠥⠃⠞⠗⠑⠑ ⠓⠁⠎ ⠁ ⠝⠥⠍⠃⠑⠗ ⠛⠗⠑⠁⠞⠑⠗ ⠞⠓⠁⠝ ⠧  ⠉⠥⠇⠁⠞⠊⠝⠛ ⠞⠓ ⠎⠏⠁⠅⠑ ⠃⠑⠗⠎ ⠓ ⠁⠝ ⠁⠝  ⠠⠑ ⠑⠗⠞⠽ ⠞⠓⠁⠇ ⠓ ⠮ ⠏⠗⠕⠏⠑⠗⠞⠽ ⠏⠁⠉⠓ ⠠⠧⠠⠑ ⠍⠃⠑⠗ ⠇⠑⠎⠎ ⠞⠓⠁⠝ ? ⠯ ⠑⠁⠉⠓ ⠧⠑⠗⠞⠑⠭ ⠠⠔  ⠠⠞⠓⠥⠎ ⠖ ⠋⠊⠝⠙ ⠧⠑⠗⠞⠑⠭ ⠉⠕⠍⠏⠁⠗⠑ ? ⠞⠓⠑⠝ ⠺ ⠓⠁⠎ ⠃⠑⠑⠝ ⠋⠕⠥⠝⠙⠲ ⠠⠊⠋⠺ ?⠗⠂ ⠞⠓⠑⠝ ⠗⠑⠏⠑⠁⠞ ⠮ ⠏⠑⠎ ⠿ ⠮ ⠇⠑⠋⠞ ⠎⠥⠃⠞⠗⠑⠑⠆ ⠊⠋ ⠺ ? ⠗ ⠗⠑⠏⠑⠁⠞ ⠮ ⠏⠗⠕⠉⠑⠎⠎ ⠿ ⠮ ⠗⠊⠛⠓⠞ ⠎⠥⠃⠞⠗⠑⠑⠲ ⠠⠃⠽⠑⠝⠞⠊ ⠗ ⠽⠺ ⠺ ⠃⠑ ⠋⠕⠥⠝⠙⠲ ⠠⠎⠥⠉⠓ ⠏⠗⠕⠏⠑⠗⠞⠊⠑⠎ ⠷ ⠞⠗⠁⠧⠑⠗ ⠎⠁⠇⠎ ⠺ ⠃⠑ ⠥⠎⠑⠙ ⠔ ⠇⠁⠞⠑⠗ ⠉⠓⠁⠏⠞⠑⠗⠎⠲ ⠓⠕⠥⠇⠙ ⠃⠑ ⠍⠁⠙⠑⠲  ⠁⠝ ⠃⠑  ⠮  ⠠⠕⠝⠑ ⠋⠊⠝⠁⠇ ⠙⠑⠋⠊⠝⠊⠞⠊⠕⠝ ⠉⠕⠝⠉⠑⠗⠝⠊⠝⠛ ⠞⠗⠑⠑⠎ ⠠⠎  ⠠⠙⠑⠋⠊⠝⠊⠞⠊⠕⠝⠲ ⠠⠁⠝ ⠥⠝⠙⠊⠗⠑⠉⠞⠑⠙ ⠞⠗⠑⠑ ⠊⠎ ⠁⠝ ⠥⠝⠙⠊⠗⠑⠉⠞⠑⠙ ⠛⠗⠁⠏⠓ ⠠⠺ ⠓⠊⠉⠓ ⠊⠎ ⠉⠕⠝⠝⠑⠉⠞⠑⠙ ⠶⠞⠓⠑⠗⠑ ⠊⠎ ⠁ ⠏⠁⠞⠓ ⠃⠑⠞⠺⠑⠑⠝ ⠁⠝⠽ ⠞⠺⠕ ⠧⠑⠗⠞⠊⠉⠑⠎⠶ ⠯ ⠁⠉⠽⠉⠇⠊⠉⠂ ⠠⠁ ⠗⠕⠕⠞⠑⠙ ⠥⠝ ⠙⠊⠗⠑⠉⠞⠑⠙ ⠞⠗⠑⠑ ⠊⠎ ⠁⠝ ⠥⠝⠙⠊⠗⠑⠉⠞⠑⠙ ⠞⠗⠑⠑ ⠔ ⠺⠓⠊⠉⠓ ⠕⠝⠑ ⠠⠧⠑⠗⠞⠑⠭ ⠊⠎ ⠙⠊⠎⠞⠊⠝⠛⠥⠊⠎⠓⠑⠙  ⠠⠎ ⠮ ⠗⠕⠕⠞  ⠠⠁ ⠙⠊⠗⠑⠉⠞⠑⠙ ⠞⠗⠑⠑ ⠉ ⠃⠑ ⠍⠁⠙⠑ ⠊⠝⠞⠕ ⠁ ⠊ ⠉ ⠃⠑ ⠍⠁ ⠼⠚ ⠁ ⠗⠕⠕⠞⠑⠙ ⠥⠝⠙⠊⠗⠑⠉⠞⠑⠙ ⠞⠗⠑⠑ ⠎⠊⠍⠏⠇⠽ ⠍⠁⠅⠊⠝⠛ ⠁⠇⠇ ⠑⠙⠛⠑⠎ ⠥⠝⠙⠊⠗⠑⠉⠞⠑⠙ ⠗⠑  ⠏⠑⠝⠇⠉⠊⠑⠞ ⠠⠺⠑ ⠎⠓⠁⠇⠇ ⠥⠎⠑ ⠮ ⠎⠁⠍⠑ ⠞⠑⠗⠍⠊⠝⠕⠇⠕⠛⠽ ⠯ ⠝⠕⠞⠁⠤ ⠍⠑⠛ ⠕⠁ ⠼⠉ ⠗⠕⠕⠞⠑⠙ ⠥⠝⠙⠊⠗⠑⠉⠞⠑⠙ ⠞⠗⠑⠑⠎ ⠵ ⠿ ⠙⠊⠗⠑⠉⠞⠑⠙ ⠞⠗⠑⠑⠎⠲ ⠠⠮ ⠏⠗⠊ ⠽ ⠍⠁⠞⠓⠑⠍⠁⠞⠊⠉⠁⠇ ⠙⠊⠎ ⠼⠁⠚⠶ ⠒ ⠼⠉ ⠧ ⠑⠊⠎⠞⠊⠝⠉⠞⠊⠕⠝ ⠊⠎ ⠞ ⠔ ⠁ ⠙⠊⠗⠑⠉⠞⠑⠙ ⠞⠗⠑⠑ ⠁⠇⠇ ⠏⠁⠞⠓⠎ ⠛ ⠋⠗ ⠼⠑  ⠠⠊ ? ?⠕⠍ ⠁⠝⠤  ⠠⠎⠑⠎⠞⠕⠗⠎ ⠖ ⠙⠑⠎⠉⠑⠝⠙⠁⠝⠞⠎ ⠺⠓⠑⠗⠑⠒ ⠠⠏⠊ ⠠⠙⠍ ⠠⠞⠑⠁⠎ ⠔ ⠁ ⠗⠕⠕⠞⠑⠙ ⠥⠝⠙⠊⠗⠑⠉⠞⠑⠙ ⠞⠗⠑⠑ ⠏⠁⠞⠓⠎ ⠑⠭⠊ ⠞⠊⠝ ⠃⠕⠞⠅ ⠠⠎? ⠕⠞⠓  ⠙⠊⠗ ⠙⠊⠗⠑⠉⠞⠊⠕⠝⠎⠲  ⠼⠃⠑ ⠠⠗⠠⠑⠠⠉⠠⠥⠠⠗⠠⠎⠠⠊⠠⠕⠠⠝  ⠠⠁ ⠏⠗⠕⠉⠑⠙⠥⠗ ⠑⠉ ⠑⠥⠗⠑ ⠑⠙⠥⠗⠑ ⠞ ⠉⠁⠇⠇⠎ ⠠⠛⠑⠎⠁⠛ ? ⠁⠇⠇⠎ ⠊⠞⠎⠑⠇⠋⠂ ⠙⠊⠗⠑⠉⠞⠇⠽ ⠷ ⠊⠝⠙⠊⠗⠑⠉⠞⠶ ⠠⠉⠠⠉⠠⠥⠠⠛ ⠠⠕⠞⠞⠑⠝⠤⠏⠑⠗ ⠍ ⠠⠭ ⠊ ⠁ ? ? ⠙⠑ ⠠⠉⠠⠞⠠⠎ ⠝⠕⠗⠑ ⠊⠝⠙ ? ⠁⠠⠇ ⠞⠑ ?⠕⠗⠑ ⠇⠥⠉⠊⠙ ⠯ ⠑⠞ ⠯ ⠉⠕⠝⠉⠊⠎⠑ ⠙⠑⠲ ⠺⠊ ⠠⠏⠕⠎⠎⠊⠃⠇⠑ ⠺⠊⠞⠓⠕⠥⠞ ⠠⠋⠑⠉⠥⠗⠎⠊⠕⠝⠲ ⠠⠔ ⠼⠁ ⠦ ⠎⠑⠑  ⠁⠊⠙ ⠼⠁⠚⠲ ⠃⠑ ⠗⠑⠉⠥⠗⠎⠊⠧⠑  ⠕⠗⠊⠞⠓⠍⠎ ⠞⠓⠁⠝ ⠞⠎  ⠠⠎⠉⠗⠊⠏⠞⠊⠕⠝⠎ ⠷ ⠃⠕⠝ ⠠⠺⠑ ⠎⠓⠁⠇⠇


SyntaxError: invalid character in identifier (<ipython-input-25-72e2538cd50c>, line 1)

In [21]:
repr(string)

"'⠉⠕⠗⠞⠑ ⠔ ⠏⠗⠑⠕⠗⠙⠑ ⠠⠍ ⠏⠗⠑⠉⠊⠎⠑⠇⠽⠒  ⠙ ⠊⠝⠁ ⠋⠊⠭⠑⠙ ⠁⠍ ⠁⠇⠕⠛⠕⠥⠎ ⠏⠗⠕⠏⠑⠗⠞⠽ ⠞ ⠑⠁⠉⠓ ⠧⠑⠗⠞⠑⠭  ⠎⠉⠑⠝⠙⠁⠝⠞ ⠷ ? ? ⠷ ⠞⠗⠑⠑ ⠎⠊⠵⠑⠲ ⠠⠏⠕⠎⠞⠕⠗⠙⠑⠗ ⠝⠥⠍⠖  ⠥⠍⠃⠑⠗⠎ ⠷ ? ⠃⠊⠝⠁⠛⠽?⠤⠞⠑⠑⠞  ⠠⠎⠕⠗⠁ ⠧⠑⠗⠞⠑⠭ ⠧ ⠓⠁⠎ ⠁ ⠝⠥⠖  ⠗⠊⠛⠓⠞ ⠎⠥⠃⠞⠗⠑⠑ ⠓⠁⠎ ⠁ ⠝⠥⠍⠃⠑⠗ ⠛⠗⠑⠁⠞⠑⠗ ⠞⠓⠁⠝ ⠧  ⠉⠥⠇⠁⠞⠊⠝⠛ ⠞⠓ ⠎⠏⠁⠅⠑ ⠃⠑⠗⠎ ⠓ ⠁⠝ ⠁⠝  ⠠⠑ ⠑⠗⠞⠽ ⠞⠓⠁⠇ ⠓ ⠮ ⠏⠗⠕⠏⠑⠗⠞⠽ ⠏⠁⠉⠓ ⠠⠧⠠⠑ ⠍⠃⠑⠗ ⠇⠑⠎⠎ ⠞⠓⠁⠝ ? ⠯ ⠑⠁⠉⠓ ⠧⠑⠗⠞⠑⠭ ⠠⠔  ⠠⠞⠓⠥⠎ ⠖ ⠋⠊⠝⠙ ⠧⠑⠗⠞⠑⠭ ⠉⠕⠍⠏⠁⠗⠑ ? ⠞⠓⠑⠝ ⠺ ⠓⠁⠎ ⠃⠑⠑⠝ ⠋⠕⠥⠝⠙⠲ ⠠⠊⠋⠺ ?⠗⠂ ⠞⠓⠑⠝ ⠗⠑⠏⠑⠁⠞ ⠮ ⠏⠑⠎ ⠿ ⠮ ⠇⠑⠋⠞ ⠎⠥⠃⠞⠗⠑⠑⠆ ⠊⠋ ⠺ ? ⠗ ⠗⠑⠏⠑⠁⠞ ⠮ ⠏⠗⠕⠉⠑⠎⠎ ⠿ ⠮ ⠗⠊⠛⠓⠞ ⠎⠥⠃⠞⠗⠑⠑⠲ ⠠⠃⠽⠑⠝⠞⠊ ⠗ ⠽⠺ ⠺ ⠃⠑ ⠋⠕⠥⠝⠙⠲ ⠠⠎⠥⠉⠓ ⠏⠗⠕⠏⠑⠗⠞⠊⠑⠎ ⠷ ⠞⠗⠁⠧⠑⠗ ⠎⠁⠇⠎ ⠺ ⠃⠑ ⠥⠎⠑⠙ ⠔ ⠇⠁⠞⠑⠗ ⠉⠓⠁⠏⠞⠑⠗⠎⠲ ⠓⠕⠥⠇⠙ ⠃⠑ ⠍⠁⠙⠑⠲  ⠁⠝ ⠃⠑  ⠮  ⠠⠕⠝⠑ ⠋⠊⠝⠁⠇ ⠙⠑⠋⠊⠝⠊⠞⠊⠕⠝ ⠉⠕⠝⠉⠑⠗⠝⠊⠝⠛ ⠞⠗⠑⠑⠎ ⠠⠎  ⠠⠙⠑⠋⠊⠝⠊⠞⠊⠕⠝⠲ ⠠⠁⠝ ⠥⠝⠙⠊⠗⠑⠉⠞⠑⠙ ⠞⠗⠑⠑ ⠊⠎ ⠁⠝ ⠥⠝⠙⠊⠗⠑⠉⠞⠑⠙ ⠛⠗⠁⠏⠓ ⠠⠺ ⠓⠊⠉⠓ ⠊⠎ ⠉⠕⠝⠝⠑⠉⠞⠑⠙ ⠶⠞⠓⠑⠗⠑ ⠊⠎ ⠁ ⠏⠁⠞⠓ ⠃⠑⠞⠺⠑⠑⠝ ⠁⠝⠽ ⠞⠺⠕ ⠧⠑⠗⠞⠊⠉⠑⠎⠶ ⠯ ⠁⠉⠽⠉⠇⠊⠉⠂ ⠠⠁ ⠗⠕⠕⠞⠑⠙ ⠥⠝ ⠙⠊⠗⠑⠉⠞⠑⠙ ⠞⠗⠑⠑ ⠊⠎ ⠁⠝ ⠥⠝⠙⠊⠗⠑⠉⠞⠑⠙ ⠞⠗⠑⠑ ⠔ ⠺⠓⠊⠉⠓ ⠕⠝⠑ ⠠⠧⠑⠗⠞⠑⠭ ⠊⠎ ⠙⠊⠎⠞⠊⠝⠛⠥⠊⠎⠓⠑⠙  ⠠⠎ ⠮ ⠗⠕⠕⠞  ⠠⠁ ⠙⠊⠗⠑⠉⠞⠑⠙ ⠞⠗⠑⠑ ⠉ ⠃⠑ ⠍⠁⠙⠑ ⠊⠝⠞⠕ ⠁ ⠊ ⠉ ⠃⠑ ⠍⠁ ⠼⠚ ⠁ ⠗⠕⠕⠞⠑⠙ ⠥⠝⠙⠊⠗⠑⠉⠞⠑⠙ ⠞⠗⠑⠑ ⠎⠊⠍⠏⠇⠽ ⠍⠁⠅⠊⠝⠛ ⠁⠇⠇ ⠑⠙⠛⠑⠎ ⠥⠝⠙⠊⠗⠑⠉⠞⠑⠙ ⠗⠑  ⠏⠑⠝⠇⠉⠊⠑⠞ ⠠⠺⠑ ⠎⠓⠁⠇⠇ ⠥⠎⠑ ⠮ ⠎⠁⠍⠑ ⠞⠑⠗⠍⠊⠝⠕⠇⠕⠛⠽ 

In [22]:
help(repr)

Help on built-in function repr in module builtins:

repr(obj, /)
    Return the canonical string representation of the object.
    
    For many object types, including most builtins, eval(repr(obj)) == obj.



In [23]:
split(string)

NameError: name 'split' is not defined

In [24]:
string.split()

['⠉⠕⠗⠞⠑',
 '⠔',
 '⠏⠗⠑⠕⠗⠙⠑',
 '⠠⠍',
 '⠏⠗⠑⠉⠊⠎⠑⠇⠽⠒',
 '⠙',
 '⠊⠝⠁',
 '⠋⠊⠭⠑⠙',
 '⠁⠍',
 '⠁⠇⠕⠛⠕⠥⠎',
 '⠏⠗⠕⠏⠑⠗⠞⠽',
 '⠞',
 '⠑⠁⠉⠓',
 '⠧⠑⠗⠞⠑⠭',
 '⠎⠉⠑⠝⠙⠁⠝⠞',
 '⠷',
 '?',
 '?',
 '⠷',
 '⠞⠗⠑⠑',
 '⠎⠊⠵⠑⠲',
 '⠠⠏⠕⠎⠞⠕⠗⠙⠑⠗',
 '⠝⠥⠍⠖',
 '⠥⠍⠃⠑⠗⠎',
 '⠷',
 '?',
 '⠃⠊⠝⠁⠛⠽?⠤⠞⠑⠑⠞',
 '⠠⠎⠕⠗⠁',
 '⠧⠑⠗⠞⠑⠭',
 '⠧',
 '⠓⠁⠎',
 '⠁',
 '⠝⠥⠖',
 '⠗⠊⠛⠓⠞',
 '⠎⠥⠃⠞⠗⠑⠑',
 '⠓⠁⠎',
 '⠁',
 '⠝⠥⠍⠃⠑⠗',
 '⠛⠗⠑⠁⠞⠑⠗',
 '⠞⠓⠁⠝',
 '⠧',
 '⠉⠥⠇⠁⠞⠊⠝⠛',
 '⠞⠓',
 '⠎⠏⠁⠅⠑',
 '⠃⠑⠗⠎',
 '⠓',
 '⠁⠝',
 '⠁⠝',
 '⠠⠑',
 '⠑⠗⠞⠽',
 '⠞⠓⠁⠇',
 '⠓',
 '⠮',
 '⠏⠗⠕⠏⠑⠗⠞⠽',
 '⠏⠁⠉⠓',
 '⠠⠧⠠⠑',
 '⠍⠃⠑⠗',
 '⠇⠑⠎⠎',
 '⠞⠓⠁⠝',
 '?',
 '⠯',
 '⠑⠁⠉⠓',
 '⠧⠑⠗⠞⠑⠭',
 '⠠⠔',
 '⠠⠞⠓⠥⠎',
 '⠖',
 '⠋⠊⠝⠙',
 '⠧⠑⠗⠞⠑⠭',
 '⠉⠕⠍⠏⠁⠗⠑',
 '?',
 '⠞⠓⠑⠝',
 '⠺',
 '⠓⠁⠎',
 '⠃⠑⠑⠝',
 '⠋⠕⠥⠝⠙⠲',
 '⠠⠊⠋⠺',
 '?⠗⠂',
 '⠞⠓⠑⠝',
 '⠗⠑⠏⠑⠁⠞',
 '⠮',
 '⠏⠑⠎',
 '⠿',
 '⠮',
 '⠇⠑⠋⠞',
 '⠎⠥⠃⠞⠗⠑⠑⠆',
 '⠊⠋',
 '⠺',
 '?',
 '⠗',
 '⠗⠑⠏⠑⠁⠞',
 '⠮',
 '⠏⠗⠕⠉⠑⠎⠎',
 '⠿',
 '⠮',
 '⠗⠊⠛⠓⠞',
 '⠎⠥⠃⠞⠗⠑⠑⠲',
 '⠠⠃⠽⠑⠝⠞⠊',
 '⠗',
 '⠽⠺',
 '⠺',
 '⠃⠑',
 '⠋⠕⠥⠝⠙⠲',
 '⠠⠎⠥⠉⠓',
 '⠏⠗⠕⠏⠑⠗⠞⠊⠑⠎',
 '⠷',
 '⠞⠗⠁⠧⠑⠗',
 '⠎⠁⠇⠎',
 '⠺',
 '⠃⠑',
 '⠥⠎⠑⠙',
 '⠔',
 '⠇⠁⠞⠑⠗',
 '⠉⠓⠁⠏⠞⠑⠗⠎⠲',
 '⠓⠕⠥⠇⠙',